In [4]:
#1. 모듈 호출
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd 
import random
import os  
from selenium.webdriver.common.keys import Keys

In [5]:
#2. 사용자 인터페이스 만들기 
print('='*100)
print('유튜브 댓글 크롤러(대댓 제외)')
print('='*100)
key_word = input('1.유튜브에서 검색할 주제 키워드를 입력하세요(예:김어준의 뉴스공장) : ')
cnt = int(input('2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇 건 인가요? : '))
d_cnt = int(input('3.각 동영상에서 추출할 댓글은 몇 건 입니까? : '))
f_dir = input('4.크롤링 결과를 저장할 폴더명만 쓰세요 : ')
if f_dir == "" : 
    f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+key_word)
os.chdir(f_dir+s+'-'+key_word)

f_name = s+'-'+key_word+'.txt'
fc_name = s+'-'+key_word+'.csv'
fx_name = s+'-'+key_word+'.xlsx'

s_time = time.time()

유튜브 댓글 크롤러(대댓 제외)
1.유튜브에서 검색할 주제 키워드를 입력하세요(예:김어준의 뉴스공장) : 다스뵈이다
2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇 건 인가요? : 3
3.각 동영상에서 추출할 댓글은 몇 건 입니까? : 50
4.크롤링 결과를 저장할 폴더명만 쓰세요 : 


In [6]:
#3. 웹드라이버 호출 & 유튜브 접속
path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
options=webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15')

driver = webdriver.Chrome(path, options=options)
driver.get('https://www.youtube.com/')
time.sleep(random.randrange(2, 5))

In [7]:
#4. 유튜브 검색창에 키워드 넣고 검색실행
element1 = driver.find_element_by_name('search_query')
element1.send_keys(key_word)
element1.send_keys('\n')
time.sleep(3)

In [8]:
#5. 링크 리스트 만들기 
link1 = []
no22 = 2
no11 = 1
if cnt <= 19 : 
    html0 = driver.page_source
    soup0 = BeautifulSoup(html0, 'html.parser')
    list0 = soup0.find_all('ytd-video-renderer', class_='style-scope ytd-item-section-renderer')
    time.sleep(3)
elif cnt > 19 : 
    #무한 스크롤 걸기
    while no22 == 2 : 
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        html0 = driver.page_source
        soup0 = BeautifulSoup(html0, 'html.parser')
        list0 = soup0.find_all('ytd-video-renderer', class_='style-scope ytd-item-section-renderer')
        time.sleep(2)
        if len(list0) >= cnt : 
            break

for a in list0 : 
    href = a.find('a', id='video-title')['href']
    link1.append(href)
    no11 += 1
    if no11 > cnt : 
        break
        
driver.quit()


In [10]:
#6. 영상 하나하나 들어가서 댓글 수집하기 
f = open(f_name, 'a', encoding='utf-8')
name1 = []
date1 = []
content1 = []
url1 = []
gogam1 = []
bgogam1 = []

a = 0
no44 = 1
no55 = 1

for b in link1 : 
    path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
    options=webdriver.ChromeOptions()
    options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')

    driver = webdriver.Chrome(path, options=options)
    driver.get('https://www.youtube.com'+b)
    time.sleep(random.randrange(2, 5))
    
    driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN) #댓글 20개 발견
    time.sleep(8)
    html1 = driver.page_source 
    soup1 = BeautifulSoup(html1, 'html.parser')
    
    


    if d_cnt <= 20 : 
        dd_list = soup1.find('div', id='contents').find_all('ytd-comment-renderer', class_='style-scope ytd-comment-thread-renderer')
    
    elif d_cnt > 20 : 
        while no22 == 2 : 
            for z in range(1, 6) : 
                driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
                time.sleep(5)
            html5 = driver.page_source
            soup5 = BeautifulSoup(html5, 'html.parser')
            dd_list = soup5.find('div', id='contents').find_all('ytd-comment-renderer', class_='style-scope ytd-comment-thread-renderer')
            time.sleep(2)
            if len(dd_list) >= d_cnt : 
                break
            elif a == len(dd_list) : 
                print('%s 개 댓글 조회를 요청하셨으나, %s 개 밖에 없어서 %s 개 만큼만 조회하겠습니다' %(d_cnt, len(dd_list), len(dd_list)))
                break
            a = len(dd_list)
            

    for c in dd_list : #하나의 댓글 안에서 
        print('%s 번째 영상의 %s 번째 댓글' %(no44,no55 ))
        f.write('%s 번째 영상의 %s 번째 댓글' %(no44, no55) +'\n')
            
        print('-'*100)
        f.write('-'*100+'\n')
            
        url = 'https://www.youtube.com'+b
        url1.append(url)
        print('1.URL 주소 :',url)
        f.write('1.URL 주소 :'+url+'\n')
            
        name = c.find('a', id='author-text').get_text().replace('\n', "").strip()
        name1.append(name)
        print('2.댓글 작성자명 :',name)
        f.write('2.댓글 작성자 명 :'+name+'\n')
            
        date = c.find('a', class_='yt-simple-endpoint style-scope yt-formatted-string').get_text().replace('\n', "").strip()
        date1.append(date)
        print('3.댓글 작성일자 :',date)
        f.write('3.댓글 작성일자 :'+date+'\n')
            
        content = c.find('yt-formatted-string', id='content-text').get_text().replace('\n', "").strip()
        content1.append(content)
        print('4.댓글 내용 :',content)
        f.write('4.댓글 내용 :'+content+'\n')
        
        gogam = c.find('span', id='vote-count-middle').get_text().replace('\n', "").strip()
        if gogam == "" : 
            gogam = " "
        gogam1.append(gogam)
        print('5.공감 횟수 :',gogam)
        f.write('5.공감 횟수 :'+gogam+'\n')
        
        bgogam = c.find('div', id='creator-heart').get_text().replace('\n', "").strip()
        if bgogam == "" : 
            bgogam = " "
        bgogam1.append(bgogam)
        print('6.크리에이터 하트 :',bgogam)
        f.write('6.크리에이터 하트 :'+bgogam+'\n')
        f.write('\n')
        print('\n')

        no55 += 1
        if no55 > d_cnt : 
            break

            
    no44 += 1
    no55 = 1
    driver.quit()
f.close()
e_time = time.time()
total_time = e_time - s_time 

1 번째 영상의 1 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=nOUmML7-T9o
2.댓글 작성자명 : 성두현
3.댓글 작성일자 : 5 days ago
4.댓글 내용 : 호사카 유지교수님 이야기를 들으니  피가 끓어오르는 같아요!!!!!!!
5.공감 횟수 : 249
6.크리에이터 하트 :  


1 번째 영상의 2 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=nOUmML7-T9o
2.댓글 작성자명 : Jirisan Partizan
3.댓글 작성일자 : 5 days ago
4.댓글 내용 : "나에게 한 발의 총알이 남아 있다면, 왜놈보다 나라와 민주주의를 배신한 매국노 변절자를 백번 천번 먼저 처단할 것이다. 왜? 왜놈보다 더 무서운 적이니까." -김구
5.공감 횟수 : 560
6.크리에이터 하트 :  


1 번째 영상의 3 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=nOUmML7-T9o
2.댓글 작성자명 : 태양처럼
3.댓글 작성일자 : 4 days ago
4.댓글 내용 : 세월호의 몸통은 김기춘, 남재준 입니다. 해경도 손발일 뿐...  김기춘, 남재준을 잡아야 합니다.
5.공감 횟수 : 9
6.크리에이터 하트 :  


1 번째 영상의 4 번째 댓글
-----------------------

50 개 댓글 조회를 요청하셨으나, 40 개 밖에 없어서 40 개 만큼만 조회하겠습니다
2 번째 영상의 1 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=f6EjePU6e1g
2.댓글 작성자명 : 딴지방송국
3.댓글 작성일자 : 1 week ago
4.댓글 내용 : 월말 김어준 웹페이지 https://m.podbbang.com/magazines/1778990/issues/1379 월말 김어준 창간호 미리듣기 https://youtu.be/hk5oiva-VA8
5.공감 횟수 : 685
6.크리에이터 하트 :  


2 번째 영상의 2 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=f6EjePU6e1g
2.댓글 작성자명 : 장재혁
3.댓글 작성일자 : 1 week ago
4.댓글 내용 : 전우용님은 정말 들을때마다 감탄이 절로 나옵니다.지식이 아니라 지혜와 해학이 넘치는 언변은.... 좀 따라하고 싶어요 ㅋㅋㅋ
5.공감 횟수 : 424
6.크리에이터 하트 :  


2 번째 영상의 3 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=f6EjePU6e1g
2.댓글 작성자명 : ELENA LEE
3.댓글 작성일자 : 1 week ago
4.댓글 내용 : 전우용 교수님 나온 부분만 따로 편집해서 올려도 좋을 거 같아요

50 개 댓글 조회를 요청하셨으나, 40 개 밖에 없어서 40 개 만큼만 조회하겠습니다
3 번째 영상의 1 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=T6SUpi6pW6A
2.댓글 작성자명 : 딴지방송국
3.댓글 작성일자 : 3 weeks ago (edited)
4.댓글 내용 : 월말 김어준 웹페이지 https://m.podbbang.com/magazines/1778990/issues/1379 월말 김어준 창간호 미리듣기 https://youtu.be/hk5oiva-VA8
5.공감 횟수 : 249
6.크리에이터 하트 :  


3 번째 영상의 2 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=T6SUpi6pW6A
2.댓글 작성자명 : 김명기
3.댓글 작성일자 : 3 weeks ago
4.댓글 내용 : 김학의를 성폭행범으로 감빵에 처넣을수 있었는데 검찰이 미적거리고 시간끌어서 공소시효지났다고 무혐의해줬는데!김학의 관련 개검찰들도 윤석열 한동훈과 같이 공수처에서 수사해라
5.공감 횟수 : 126
6.크리에이터 하트 :  


3 번째 영상의 3 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=T6SUpi6pW6A
2.댓글 작성자명 : August
3.댓글 작성일자 : 3 weeks ago
4.댓글 내용 : 뉴

In [11]:
#7. csv, xlsx 파일로 저장하기 
you = pd.DataFrame()
you['URL 주소'] = url1
you['댓글작성자명'] = name1
you['댓글작성일자'] = date1
you['댓글내용'] = content1
you['공감횟수'] = gogam1
you['크리에이터 하트'] = bgogam1

you.to_csv(fc_name, encoding='utf-8-sig')
you.to_excel(fx_name)

print('='*100)
print('총 %s 개 영상의 댓글을 수집하였습니다' %(cnt))
print('총 소요된 시간은 %s 초 입니다' %round(total_time, 1))
print('='*100)


총 3 개 영상의 댓글을 수집하였습니다
총 소요된 시간은 220.1 초 입니다


19